In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Email dataset
email_data = [
    ("Can I get the syllabus for the Machine Learning course?", 0),
    ("When is the exam of deep learning course?", 0),  # Added missing comma
    ("We would like to collaborate on a research project.", 1),
    ("Are there internship opportunities in your department?", 2),
    ("Could you provide the timetable for next semester?", 0),
    ("Looking for data access to research facilities.", 1),
    ("Can you help me with the assignment deadline?", 0),
    ("We are interested in conducting a workshop at IITGN.", 2),
    ("How can I register for the upcoming semester?", 0),
    ("We want to sponsor a project with your research group.", 1),
    ("Can I apply for a placement drive in your department?", 2),
]

# Labels: 0 -> Student, 1 -> Research, 2 -> Corporate

# Preprocess email data
emails = [email[0] for email in email_data]
labels = [email[1] for email in email_data]

# Vectorize the email using TF-IDF (NLP)
vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
X = vectorizer.fit_transform(emails)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

# Define the Email Classifier model
class EmailClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(EmailClassifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        return self.fc(x)

# Train the model
def train_model(X_train, y_train, input_size, num_classes):
    model = EmailClassifier(input_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    X_train = torch.FloatTensor(X_train.toarray())
    y_train = torch.LongTensor(y_train)

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    print(f'Training completed with final loss: {loss.item()}')
    return model

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    model.eval()
    X_test = torch.FloatTensor(X_test.toarray())
    y_test = torch.LongTensor(y_test)

    with torch.no_grad():
        predictions = model(X_test)
        predicted_labels = torch.argmax(predictions, dim=1)
        accuracy = accuracy_score(y_test, predicted_labels)
    
    print(f'Model accuracy: {accuracy}')

# Automated response generator
def fetch_response(query):
    responses = {
        'syllabus': 'Here is the syllabus for your requested course: [link]',
        'exam': 'The exam is on 20th Oct.',
        'timetable': 'The department schedule can be accessed here: [link]',
        'internship': 'Yes, we have internship opportunities available. Please contact us for more details.',
        'research': 'Thank you for your interest in research collaboration. We will get back to you shortly.'
    }

    for keyword, response in responses.items():
        if keyword in query.lower():
            return response
    
    return "Sorry, request not processed. It has been escalated."

def generate_response(email_text):
    response = fetch_response(email_text)
    return response

# Classify and handle an email
def classify_email(model, vectorizer, email_text):
    email_vector = vectorizer.transform([email_text])
    with torch.no_grad():
        prediction = model(torch.FloatTensor(email_vector.toarray()))
        label = torch.argmax(prediction, dim=1).item()
    return label

def handle_email(model, vectorizer, email_text):
    label = classify_email(model, vectorizer, email_text)
    
    if label == 0:  # Student inquiries
        response = generate_response(email_text)
    elif label == 1:  # Academic collaboration
        response = "Your research query has been forwarded to the respective HOD."
    elif label == 2:  # Corporate inquiries
        response = "Your corporate inquiry has been escalated to the respective HOD."
    else:
        response = "Your email is being processed."

    return response

# Main execution
if __name__ == "__main__":
    input_size = X_train.shape[1]
    num_classes = 3  # Student, Research, Corporate

    # Train and evaluate the model
    model = train_model(X_train, y_train, input_size, num_classes)
    evaluate_model(model, X_test, y_test)

    # Example: Handle a sample email
    sample_email = "I would like to request the syllabus for the Data Structures course."
    response = handle_email(model, vectorizer, sample_email)
    print(f"Response: {response}")

Training completed with final loss: 0.8743261694908142
Model accuracy: 0.75
Response: Here is the syllabus for your requested course: [link]
